## 범주형 변수 변환 방법
1. 더미화
  - 가장 일반적인 범주형 변수를 변환하는 방법으로, 범주형 변수가 특정 값을 취하는지 여부를 나타내는 더미 변수를 생성하는 방법
2. 연속형 변수로 치환
  - 범주형 변수의 상태 공간 크기가 클 때, 더미화는 과하게 많은 변수를 추가해서 차원의 저주 문제로 이어질 수 있음
  - 라벨 정보를 활용하여 범주 변수를 연속형 변수로 치환하면 기존 변수가 가지는 정보가 일부 손실될 수 있고 활용이 어렵다는 단점이 있으나, 차원의 크기가 변하지 않으며 더 효율적인 변수로 변환할 수 있다는 장점이 있음

### feature_engine.categorical_encoders.OneHotCategoricalEncoder
- 더미화를 하기 위한 함수로, 활용 방법은 sklearn의 인스턴스의 활용 방법과 유사함
- 주요 입력
  - variables: 더미화 대상이 되는 범주형 변수의 이름 목록 (주의 : 해당 변수는 반드시 str타입이어야 함)
  - drop_last : 한 범주 변수로부터 만든 더미 변수 가운데 마지막 더미 변수를 제거할 지를 결정
  - top_categories : 한 범주 변수로 부터 만드는 더미 변수 개수를 설정하며, 빈도 기준으로 자름
- 참고 : pandas.get_dummies()는 이 함수보다 사용이 훨씬 간단하지만, 학습 데이터에 포함된 범주형 변수를 처리한 방식으로 새로 들어온 데이터에 적용이 불가능하기 때문에, 실제적으로 활용이 어려움

In [1]:
import os
import pandas as pd
os.chdir(r'C:\Users\신상훈\Desktop\preprocessing\part-4.-머신러닝을-위한-필수-전처리\Part 4. 머신러닝을 위한 필수 전처리\데이터')

In [2]:
df = pd.read_csv('car-good.csv')

In [3]:
# 특징과 라벨 분리
X = df.drop('Class', axis = 1)
Y = df['Class']

In [4]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [5]:
Train_Y.value_counts()

negative    629
positive     19
Name: Class, dtype: int64

In [6]:
# 문자 라벨을 숫자로 치환
Train_Y.replace({'negative':-1,'positive':1},inplace= True)
Test_Y.replace({'negative':-1,'positive':1},inplace= True)

In [7]:
Train_X.head()

,Buying,Maint,Doors,Persons,Lug_boot,Safety
698,low,vhigh,4,4,med,high
554,med,med,2,4,med,high
784,low,med,3,4,small,med
32,vhigh,vhigh,3,4,med,high
728,low,high,3,2,big,high


In [8]:
# 자세한 범주형 변수 판별 => 모든 변수가 범주형임을 확인
for col in Train_X.columns:
    print(col, len(Train_X[col].unique()))

Buying 4
Maint 4
Doors 3
Persons 2
Lug_boot 3
Safety 3


## 더미화를 이용한 범주 변수 처리

In [9]:
Train_X = Train_X.astype(str) # 모든 변수가 범주이므로, 더미화를 위해 전부 string 타입으로 변환

In [13]:
from feature_engine.encoding import OneHotEncoder as OHE

dummy_model = OHE(variables = Train_X.columns.tolist(),
                 drop_last = True)

dummy_model.fit(Train_X)

d_Train_X = dummy_model.transform(Train_X)
d_Test_X = dummy_model.transform(Test_X)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [14]:
# 더미화를 한 뒤의 모델 테스트
from sklearn.neighbors import KNeighborsClassifier as KNN
model = KNN().fit(d_Train_X, Train_Y)
pred_Y = model.predict(d_Test_X)

from sklearn.metrics import f1_score
f1_score(Test_Y, pred_Y)

0.0

In [21]:
d_Train_X

,Buying_low,Buying_med,Buying_vhigh,Maint_vhigh,Maint_med,Maint_high,Doors_4,Doors_2,Persons_4,Lug_boot_med,Lug_boot_small,Safety_high,Safety_med
698,1,0,0,1,0,0,1,0,1,1,0,1,0
554,0,1,0,0,1,0,0,1,1,1,0,1,0
784,1,0,0,0,1,0,0,0,1,0,1,0,1
32,0,0,1,1,0,0,0,0,1,1,0,1,0
728,1,0,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,0,1,0,0,0,1,1,0,1,0,1,1,0
83,0,0,1,0,0,1,0,0,1,0,1,1,0
291,0,0,0,0,0,1,0,0,0,1,0,0,0
416,0,0,0,0,0,0,1,0,0,0,1,1,0


In [16]:
d_Train_X['Buying_low'].unique()

array([1, 0])

#### 연속형 변수로 치환

In [17]:
Train_df = pd.concat([Train_X, Train_Y], axis = 1)
for col in Train_X.columns: # 보통은 범주 변수만 순회
    temp_dict = Train_df.groupby(col)['Class'].mean().to_dict() # col에 따른 Class의 평균을 나타내는 사전 (replace를 쓰기 위해, 사전으로 만듦)
    Train_df[col] = Train_df[col].replace(temp_dict) # 변수 치환    
    Test_X[col] = Test_X[col].astype(str).replace(temp_dict) # 테스트 데이터도 같이 치환해줘야 함 (나중에 활용하기 위해서는 저장도 필요)

<ipython-input-17-a39f41943cb6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_X[col] = Test_X[col].astype(str).replace(temp_dict) # 테스트 데이터도 같이 치환해줘야 함 (나중에 활용하기 위해서는 저장도 필요)


In [18]:
Train_df.head()

,Buying,Maint,Doors,Persons,Lug_boot,Safety,Class
698,-0.860759,-1.000,-0.952830,-0.881988,-0.944700,-0.885965,-1
554,-0.900000,-0.925,-0.944186,-0.881988,-0.944700,-0.885965,-1
784,-0.860759,-0.925,-0.927602,-0.881988,-0.918552,-0.942857,-1
32,-1.000000,-1.000,-0.927602,-0.881988,-0.944700,-0.885965,-1
728,-0.860759,-1.000,-0.927602,-1.000000,-0.961905,-0.885965,-1


In [19]:
Train_X = Train_df.drop('Class', axis = 1)
Train_Y = Train_df['Class']

In [20]:
# 치환한 뒤의 모델 테스트
model = KNN().fit(Train_X, Train_Y)
pred_Y = model.predict(Test_X)

f1_score(Test_Y, pred_Y)


# 라벨을 고려한 전처리이므로 더미화보다 좋은 결과가 나왔음 => 차원도 줄고 성능 상에 이점이 있으나, 

0.5714285714285715